In [3]:
!nvidia-smi -L || echo "No GPU detected (ok for quick tests)"
!pip -q install "transformers==4.44.0" "datasets==3.0.1" "torch" "sentencepiece" \
                 "pandas" "numpy" "matplotlib" "tqdm" "evaluate"

/bin/bash: line 1: nvidia-smi: command not found
No GPU detected (ok for quick tests)


In [8]:
import pandas as pd
import numpy as np
import re

CSV_PATH = "/content/Round24_AbuDhabi.csv"

df_raw = pd.read_csv(CSV_PATH)
print("Columns:", df_raw.columns.tolist())
df_raw.head(5)

Columns: ['post_timestamp', 'post_title', 'post_content']


,post_timestamp,post_title,post_content
0,2024-12-08 22:00:00,러셀 몽골 레슬링 ㅋㅋㅋ,존버
1,2024-12-08 22:00:01,오늘 몽골이 맥라렌 두대 박으면 엪갤 터지냐,NaN
2,2024-12-08 22:00:04,열차짤 만든 엪붕이 좋아죽겠노 ㅋㅋㅋㅋㅋㅋㅋ,안캐 샤라웃 ㅋㅋㅋㅋ\n- dc official App
3,2024-12-08 22:00:08,오늘 준비 엄청 하셨네ㅋㅋㅋㅋㅋㅋ,- dc official App
4,2024-12-08 22:00:13,야스 마리나 서킷이 전형적인 틸케 드롬은 아니냐?,아니면 결국 좆노잼 병신이냐?


In [9]:
# 후보 컬럼들
TEXT_CAND = ["text","content","message","body","comment","chat"]
TIME_CAND = ["timestamp","time","created_at","createdAt","date","datetime"]
GROUP_CAND = ["team","team_name","driver","player","user","author"]  # 분석용 옵션

def pick_col(cols, cand_list):
    # 1) 정확히 일치하는 이름 우선
    for c in cand_list:
        if c in cols:
            return c
    # 2) 부분적으로 포함되는 이름 (대소문자 무시)
    for col in cols:
        for k in cand_list:
            if k.lower() in col.lower():
                return col
    return None

TEXT_COL  = pick_col(df_raw.columns, TEXT_CAND)
TIME_COL  = pick_col(df_raw.columns, TIME_CAND)
GROUP_COL = pick_col(df_raw.columns, GROUP_CAND)

print(f"Detected -> TEXT_COL={TEXT_COL}, TIME_COL={TIME_COL}, GROUP_COL={GROUP_COL}")

if TEXT_COL is None:
    raise ValueError("텍스트(채팅) 컬럼을 못 찾았어요. df_raw.rename(columns={'채팅컬럼명':'text'}) 하고 다시 실행해주세요.")

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    # URL 제거
    s = re.sub(r"http\S+", " ", s)
    # 멘션/해시태그 제거
    s = re.sub(r"[@#]\w+", " ", s)
    # 반복 문자 축약 (ㅋㅋㅋㅋ -> ㅋㅋ, 아아아아아 -> 아아)
    s = re.sub(r"([ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9!?.])\1{2,}", r"\1\1", s)
    # 이모지/특수문자 과도한 것 정리 (한글/영문/숫자/기본문장부호만 유지)
    s = re.sub(r"[^\w\s가-힣ㄱ-ㅎㅏ-ㅣ!?.]", " ", s)

    # 팀명/드라이버 약칭 정규화 (원하는 만큼 추가 가능)
    team_map = {
        "T1": "티원",
        "GEN": "젠지",
        "VER": "페르스타펜",
        "HAM": "해밀턴",
        "ALO": "알론소",
        "NOR": "노리스",
        "LEC": "르클레르"
    }
    for short_name, full_name in team_map.items():
        s = re.sub(fr"\b{short_name}\b", full_name, s, flags=re.IGNORECASE)

    # 공백 정리
    s = re.sub(r"\s+", " ", s).strip()
    return s

df = df_raw.copy()
df["clean_text"] = df[TEXT_COL].astype(str).map(normalize_text)

# 타임스탬프 파싱 (있으면)
if TIME_COL is not None:
    df["ts"] = pd.to_datetime(df[TIME_COL], errors="coerce")
else:
    df["ts"] = pd.NaT

# 빈 텍스트 제거
df = df.dropna(subset=["clean_text"]).reset_index(drop=True)

print("After cleaning, rows:", len(df))
df[["clean_text","ts"]].head(10)

Detected -> TEXT_COL=post_content, TIME_COL=post_timestamp, GROUP_COL=None
After cleaning, rows: 5577


,clean_text,ts
0,존버,2024-12-08 22:00:00
1,nan,2024-12-08 22:00:01
2,안캐 샤라웃 ㅋㅋ dc official App,2024-12-08 22:00:04
3,dc official App,2024-12-08 22:00:08
4,아니면 결국 좆노잼 병신이냐?,2024-12-08 22:00:13
5,가만보면 막스가 맨날 막타에 나오는듯해서,2024-12-08 22:00:24
6,갤에서 한캐 여론 좋으니까 준비 빡세게 해온거 티납니다,2024-12-08 22:00:26
7,캬ㅋㅋ,2024-12-08 22:00:36
8,미치겠네 진짜 ㅋㅋ,2024-12-08 22:00:39
9,ㄹㅇ,2024-12-08 22:00:40


In [10]:
from datasets import load_dataset

# NSMC: 영화 리뷰 긍/부정. label은 0(부정)/1(긍정)
# datasets.load_dataset("csv", ...) 방식으로 직접 TSV를 읽어온다
nsmc = load_dataset(
    "csv",
    data_files={
        "train": "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
        "test":  "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
    },
    delimiter="\t"
)

print(nsmc)
nsmc["train"][0]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


{'id': 9976970, 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
import evaluate
import torch
import numpy as np

# 모델 후보. 첫 번째가 잘 불리면 그대로 사용.
BASES = ["beomi/KcELECTRA-base", "beomi/KcELECTRA-base-v2022"]

tok = None
MODEL_BASE = None
for base in BASES:
    try:
        tok = AutoTokenizer.from_pretrained(base)
        MODEL_BASE = base
        print("Using base:", base)
        break
    except Exception as e:
        print("Tokenizer load failed:", base, "->", e)

if tok is None:
    raise RuntimeError("KcELECTRA 토크나이저 로드 실패")

def tok_fn(batch):
    # 앞서 클리닝한 df 데이터프레임의 "clean_text" 컬럼을 인풋 텍스트로 사용
    # 비문자열 값을 문자열로 변환하여 처리
    texts = [str(text) for text in batch["clean_text"]]
    out = tok(texts, truncation=True, max_length=128)
    # 라벨이 있다면 라벨 컬럼도 처리 (여기서는 라벨 컬럼이 없으므로 제거)
    # out["labels"] = batch["label"] # NSMC 데이터셋의 라벨 컬럼은 여기서는 사용하지 않음
    return out

# nsmc 데이터셋 대신 클리닝된 df 데이터프레임을 사용
# 데이터프레임을 Dataset 객체로 변환
from datasets import Dataset
df_dataset = Dataset.from_pandas(df)

tokenized = df_dataset.map(
    tok_fn,
    batched=True,
    # remove_columns는 더 이상 nsmc 데이터셋을 사용하지 않으므로 필요 없습니다.
    # remove_columns=remove_cols
)

collator = DataCollatorWithPadding(tokenizer=tok)
metric_f1 = evaluate.load("f1")

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_BASE,
    num_labels=2 # 이진 분류 (긍정/부정)를 위한 num_labels=2
)

print("Tokenized dataset structure:", tokenized)

Using base: beomi/KcELECTRA-base


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/5577 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenized dataset structure: Dataset({
    features: ['post_timestamp', 'post_title', 'post_content', 'clean_text', 'ts', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5577
})


In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
import evaluate
import torch
import numpy as np
from tqdm.auto import tqdm # tqdm 라이브러리를 가져옵니다.

# 모델 후보. 첫 번째가 잘 불리면 그대로 사용.
BASES = ["beomi/KcELECTRA-base", "beomi/KcELECTRA-base-v2022"]

tok = None
MODEL_BASE = None
for base in BASES:
    try:
        tok = AutoTokenizer.from_pretrained(base)
        MODEL_BASE = base
        print("Using base:", base)
        break
    except Exception as e:
        print("Tokenizer load failed:", base, "->", e)

if tok is None:
    raise RuntimeError("KcELECTRA 토크나이저 로드 실패")

# NSMC 데이터셋 로드
from datasets import load_dataset
nsmc = load_dataset(
    "csv",
    data_files={
        "train": "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
        "test":  "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
    },
    delimiter="\t"
)

# 수동으로 데이터 토큰화
tokenized_inputs = []
batch_size = 64 # 원하는 배치 크기 설정

# NSMC 훈련 데이터셋의 'document' 컬럼 사용
texts = [str(text) for text in nsmc["train"]["document"]]
labels = nsmc["train"]["label"]

for i in tqdm(range(0, len(texts), batch_size), desc="Tokenizing"):
    batch_texts = texts[i : i + batch_size]
    batch_labels = labels[i : i + batch_size]

    enc = tok(
        batch_texts,
        truncation=True,
        max_length=128,
        padding="max_length",
        return_tensors="pt"
    )
    enc["labels"] = torch.tensor(batch_labels) # 라벨을 텐서로 변환하여 추가
    tokenized_inputs.append(enc)

# 토큰화된 배치를 하나의 딕셔너리로 합칩니다.
# 실제 훈련 시에는 DataCollator를 사용하여 배치별로 처리하게 됩니다.
# 여기서는 구조 확인을 위해 첫 번째 배치를 출력합니다.
print("\nFirst tokenized batch structure:", tokenized_inputs[0].keys())
print("Input IDs shape:", tokenized_inputs[0]["input_ids"].shape)
print("Labels shape:", tokenized_inputs[0]["labels"].shape)


collator = DataCollatorWithPadding(tokenizer=tok)
metric_f1 = evaluate.load("f1")

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_BASE,
    num_labels=2
)

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Using base: beomi/KcELECTRA-base


Tokenizing:   0%|          | 0/2344 [00:00<?, ?it/s]


First tokenized batch structure: dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
Input IDs shape: torch.Size([64, 128])
Labels shape: torch.Size([64])


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="./results",          # 훈련 결과 및 체크포인트 저장 디렉토리
    evaluation_strategy="epoch",     # 에포크마다 평가 수행
    save_strategy="epoch",           # 에포크마다 모델 저장
    learning_rate=2e-5,              # 학습률
    per_device_train_batch_size=16,  # 장치당 훈련 배치 크기
    per_device_eval_batch_size=16,   # 장치당 평가 배치 크기
    num_train_epochs=3,              # 훈련 에포크 수
    weight_decay=0.01,               # 가중치 감소 (L2 정규화)
    push_to_hub=False,               # Hugging Face Hub에 푸시하지 않음
    report_to="none",                # 로깅 비활성화
)

# 평가 메트릭 함수 정의
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric_f1.compute(predictions=predictions, references=labels)

# Trainer 인스턴스 생성
# 훈련 데이터셋은 수동으로 토큰화한 tokenized_inputs 리스트를 사용합니다.
# 검증 데이터셋은 NSMC 테스트 데이터셋을 사용하여 토큰화해야 합니다.

# NSMC 테스트 데이터셋 토큰화
test_texts = [str(text) for text in nsmc["test"]["document"]]
test_labels = nsmc["test"]["label"]

tokenized_test_inputs = []
for i in tqdm(range(0, len(test_texts), batch_size), desc="Tokenizing Test Data"):
    batch_test_texts = test_texts[i : i + batch_size]
    batch_test_labels = test_labels[i : i + batch_size]

    enc = tok(
        batch_test_texts,
        truncation=True,
        max_length=128,
        padding="max_length",
        return_tensors="pt"
    )
    enc["labels"] = torch.tensor(batch_test_labels)
    tokenized_test_inputs.append(enc)

# tokenized_inputs는 리스트 형태이므로 Trainer에 전달하기 위해 Dataset 객체로 변환
# 이 부분은 Trainer가 Dataset 객체를 받도록 설계되어 있어 필요합니다.
# 각 딕셔너리를 개별 샘플로 변환하여 리스트로 만들고, 이를 Dataset.from_list로 변환
train_data_list = []
for batch in tokenized_inputs:
    for i in range(batch["input_ids"].shape[0]):
        train_data_list.append({
            "input_ids": batch["input_ids"][i].tolist(),
            "token_type_ids": batch["token_type_ids"][i].tolist(),
            "attention_mask": batch["attention_mask"][i].tolist(),
            "labels": batch["labels"][i].item()
        })

test_data_list = []
for batch in tokenized_test_inputs:
    for i in range(batch["input_ids"].shape[0]):
         test_data_list.append({
            "input_ids": batch["input_ids"][i].tolist(),
            "token_type_ids": batch["token_type_ids"][i].tolist(),
            "attention_mask": batch["attention_mask"][i].tolist(),
            "labels": batch["labels"][i].item()
        })

from datasets import Dataset
train_dataset = Dataset.from_list(train_data_list)
test_dataset = Dataset.from_list(test_data_list)


trainer = Trainer(
    model=model,                         # 훈련할 모델
    args=training_args,                  # 훈련 인자
    train_dataset=train_dataset,         # 훈련 데이터셋
    eval_dataset=test_dataset,           # 검증 데이터셋
    data_collator=collator,              # 데이터 콜레이터
    compute_metrics=compute_metrics,     # 평가 메트릭 함수
    tokenizer=tok                        # 토크나이저
)

# 훈련 시작
print("Starting training...")
trainer.train()
print("Training finished.")

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Tokenizing Test Data:   0%|          | 0/782 [00:00<?, ?it/s]

Starting training...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


모델 훈련이 완료되면 `./results` 디렉토리에 저장됩니다. 이제 훈련된 모델을 사용하여 `df` 데이터프레임의 텍스트에 대한 감성 분석을 수행할 수 있습니다.